In [ ]:
!pip install deeprobust

     |████████████████████████████████| 184 kB 7.1 MB/s 
     |████████████████████████████████| 24.2 MB 100 kB/s 
     |████████████████████████████████| 124 kB 70.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import numpy as np
import scipy.sparse as sp
from copy import deepcopy
from sklearn.metrics import f1_score
from deeprobust.graph.data import Dataset
from tqdm import tqdm
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from tqdm import tqdm

No module named 'torch_geometric'


/usr/local/lib/python3.7/dist-packages/deeprobust/graph/data/__init__.py:11: UserWarning: Please install pytorch geometric if you would like to use the datasets from pytorch geometric. See details in https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
  "geometric. See details in https://pytorch-geom" +


# UTILS

In [ ]:
def classification_margin(output,true_label):
  '''
  Output 에 대한 classification margin 값을 구합니다.
  probs_true_label - probs_best_second_class

  Parameters :
  1. output 
    자료형 : torch.Tensor
    설명 : output vector (1차원)
  
  2. true_label
    자료형 : int
    설명 : 해당 노드의 true label 
  
  Return값 :
  1. list
    설명 : 해당 노드의 classification margin

  '''
  probs = torch.exp(output)
  probs_true_label = probs[true_label].clone()
  probs[true_label] = 0 # true label의 probs값은 0으로 만들어놓고 argmax를 통해 두번째로 probs가 높은 값을 찾는다.
  probs_best_second_class = probs[probs.argmax()]
  return (probs_true_label - probs_best_second_class).item()

In [ ]:
def normalize_adj(mx) : 
  '''
  희소 인접 행렬(Sparse adjacency matrix)을 Normalize 합니다.
  A' = (D+I)^(-1/2) * (A+I)*(D+I)^(-1/2)

  Parameters : 
  1. mx
    자료형 : scipy.sparse.csr_matrix
    설명 : normalize 할 matrix
  
  Returns : 
  자료형 : scipy.sparse.lil_matrix
  설명 : normalize 된 matrix
  '''

  # TODO : coo format을 사용하는 것이 더 나을지도?
  if type(mx) is not sp.lil.lil_matrix:
    mx = mx.tolil()
  if mx[0,0] == 0:
    # 만약 I 행렬이 더해져 있지 않은 상태이면 I 행렬을 더해줌(self-loop)
    mx = mx+sp.eye(mx.shape[0])
  rowsum = np.array(mx.sum(1))
  r_inv = np.power(rowsum, -1/2).flatten()
  r_inv[np.isinf(r_inv)] = 0.
  r_mat_inv = sp.diags(r_inv)
  mx = r_mat_inv.dot(mx)
  mx = mx.dot(r_mat_inv)
  return mx

In [ ]:
def to_scipy(tensor):
  ''' Dense / Sparse tensor를 scipy matrix로 변환합니다'''
  if is_sparse_tensor(tensor):
    values = tensor._values()
    indices = tensor._indices()
    return sp.csr_matrix((values.cpu().numpy(), indices.cpu().numpy()), shape = tensor.shape)
  else:
    indices = tensor.nonzero().t()
    values = tensor[indices[0], indices[1]]
    return sp.csr_matrix((values.cpu().numpy(), indices.cpu().numpy()), shape = tensor.shape)

In [ ]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
  ''' scipy sparse matrix를 torch sparse tensor로 변환합니다'''
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  sparserow = torch.LongTensor(sparse_mx.row).unsqueeze(1)
  sparsecol = torch.LongTensor(sparse_mx.col).unsqueeze(1)
  sparseconcat = torch.cat((sparserow, sparsecol),1)
  sparsedata = torch.FloatTensor(sparse_mx.data)
  return torch.sparse.FloatTensor(sparseconcat.t(), sparsedata, torch.Size(sparse_mx.shape))

In [ ]:
def to_tensor(adj, features, labels = None, device = 'cpu'):
  '''
  인접행렬(adj), Feature행렬(features), 라벨(labels)들을 array 혹은 sparse matrix에서 
  torch Tensor 로 바꿔줍니다.

  Parameters : 
  1. adj
    자료형 : scipy.sparse.csr_matrix
    설명 : 인접행렬
  2. features
    자료형 : scipy .sparse.csr_matrix
    설명 : 노드 features
  3. labels
    자료형 : numpy.array
    설명 : 노드 라벨
  4. device
    자료형 : str
    설명 : cpu or cuda
  '''
  # Adj matrix 변환
  if sp.issparse(adj):
    adj = sparse_mx_to_torch_sparse_tensor(adj)
  else:
    adj = torch.FloatTensor(adj)
  
  # Feature matrix 변환
  if sp.issparse(features):
    features = sparse_mx_to_torch_sparse_tensor(features)
  else:
    features = torch.FloatTensor(features)
  
  # label 변환
  if labels is None:
    return adj.to(device), features.to(device)
  else:
    labels = torch.LongTensor(labels)
    return adj.to(device), features.to(device), labels.to(device)
  

In [ ]:
def normalize_adj_tensor(adj, sparse=False):
  ''' Adjacency tensor matrix를 normalize 합니다'''
  device = torch.device('cuda' if adj.is_cuda else 'cpu')
  if sparse:
    adj = to_scipy(adj)
    mx = normalize_adj(adj)
    return sparse_mx_to_torch_sparse_tensor(mx).to(device)
  else:
    mx = adj+torch.eye(adj.shape[0]).to(device)
    rowsum = mx.sum(1)
    r_inv = rowsum.pow(-1/2).flatten()
    r_inv[torch.isinf(r_inv)]=0.
    r_mat_inv = torch.diag(r_inv)
    mx = r_mat_inv @ mx
    mx = mx @ r_mat_inv
  return mx

In [ ]:
def get_train_val_test(nnodes, val_size=0.1, test_size=0.8, stratify = None, seed=None):
  '''
  Nettack / Mettack의 설정값을 따릅니다.
  노드 중 10%는 training, 10%는 validation, 나머지 80%는 test용으로 사용합니다.

  Parameters:
  1. nnodes
    자료형 : int
    설명 : 총 노드 개수
  2. val_size
    자료형 : float
    설명 : validation set의 size
  3. test_size 
    자료형 : float
    설명 : test set의 size
  4. stratify
    설명 : data가 stratified fashion으로 나뉘어지게 될 것입니다. 
          따라서 stratify는 label이 되어야 합니다.
  5. seed 
    자료형 : int or None
    설명 : random seed

  Returns :
  1. idx_train
    설명 : training node의 인덱스
  2. idx_val
    설명 : validation node의 인덱스
  3. idx_test
    설명 : test set의 인덱스
  '''

  assert stratify is not None, 'stratify cannot be None!'

  if seed is not None:
        np.random.seed(seed)

  idx = np.arange(nnodes)
  train_size = 1 - val_size - test_size
  idx_train_and_val, idx_test = train_test_split(idx,
                                                   random_state=None,
                                                   train_size=train_size + val_size,
                                                   test_size=test_size,
                                                   stratify=stratify)

  if stratify is not None:
      stratify = stratify[idx_train_and_val]

  idx_train, idx_val = train_test_split(idx_train_and_val,
                                          random_state=None,
                                          train_size=(train_size / (train_size + val_size)),
                                          test_size=(val_size / (train_size + val_size)),
                                          stratify=stratify)

  return idx_train, idx_val, idx_test

In [ ]:
def get_train_val_test_gcn(labels, seed=None):
  '''
  GCN setting을 따릅니다. 각 class별로 20개의 인스턴스를 training data로 랜덤하게
  샘플링하고, 500개의 인스턴스를 validation data로, 1000개의 인스턴스를 test data로
  샘플링합니다. Random seed가 바뀌면 split도 바뀌게 됩니다.

  Parameters : 
  1. labels
    자료형 : numpy.array
    설명 : 노드 라벨
  2. seed
    자료형 : int 혹은 None
    설명 : random seed

  Returns : 
  1. idx_train
    설명 : training node 인덱스
  2. idx_val
    설명 : validation node 인덱스
  3. idx_test
    설명 : test node 인덱스
  '''

  if seed is not None:
      np.random.seed(seed)

  idx = np.arange(len(labels))
  nclass = labels.max() + 1
  idx_train = []
  idx_unlabeled = []
  for i in range(nclass):
      labels_i = idx[labels==i]
      labels_i = np.random.permutation(labels_i)
      idx_train = np.hstack((idx_train, labels_i[: 20])).astype(np.int)
      idx_unlabeled = np.hstack((idx_unlabeled, labels_i[20: ])).astype(np.int)

  idx_unlabeled = np.random.permutation(idx_unlabeled)
  idx_val = idx_unlabeled[: 500]
  idx_test = idx_unlabeled[500: 1500]
  return idx_train, idx_val, idx_test


In [ ]:
def is_sparse_tensor(tensor):
  '''
  tensor가 sparse tensor 인지 확인합니다.

  Parameters : 
  1. tensor
    자료형 : torch.Tensor
    설명 : 주어진 tensor

  Returns : 
  1. bool : tensor가 sparse tensor인지 아닌지에 대한 결과

  '''
  if tensor.layout == torch.sparse_coo:
    return True
  else:
    return False

In [ ]:
def accuracy(output, labels):
  '''
  output과 label을 비교했을 때의 accuracy값을 도출합니다.

  Parameters : 
  1. output 
    자료형 : torch.Tensor
    설명 : 모델에서 나온 output 결과
  2. labels
    자료형 : torch.Tensor 혹은 numpy.array
    설명 : 노드 라벨

  Returns : 
  1. float : accuray 정보

  '''
  if not hasattr(labels, '__len__'):
      labels = [labels]
  if type(labels) is not torch.Tensor:
      labels = torch.LongTensor(labels)
  preds = output.max(1)[1].type_as(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

# BaseAttack

In [ ]:
class BaseAttack(Module):
    """Abstract base class for target attack classes.

    Parameters
    ----------
    model :
        model to attack
    nnodes : int
        number of nodes in the input graph
    attack_structure : bool
        whether to attack graph structure
    attack_features : bool
        whether to attack node features
    device: str
        'cpu' or 'cuda'

    """

    def __init__(self, model, nnodes, attack_structure=True, attack_features=False, device='cpu'):
        super(BaseAttack, self).__init__()

        self.surrogate = model
        self.nnodes = nnodes
        self.attack_structure = attack_structure
        self.attack_features = attack_features
        self.device = device

        if model is not None:
            self.nclass = model.nclass
            self.nfeat = model.nfeat
            self.hidden_sizes = model.hidden_sizes

        self.modified_adj = None
        self.modified_features = None

    def attack(self, ori_adj, n_perturbations, **kwargs):
        """
        입력된 Graph에 대해 perturbation을 가합니다.

        Parameters:
        1. ori_adj
          자료형 : scipy.sparse.csr_matrix
          설명 : 원래(unperturbed) 인접행렬
        2. n_perturbations
          자료형 : int
          설명 : Input graph에 대한 perturbation 개수. 
                 Perturbation은 간선 제거/추가 혹은 feature 제거/추가가 될 수 있다.
        
        Returns : 
        없음

        """
        pass


    def check_adj(self, adj):
      '''
      인접행렬이 대칭이고 가중치가 없는지 확인합니다.
      '''

      if type(adj) is torch.Tensor:
          adj = adj.cpu().numpy()
      assert np.abs(adj - adj.T).sum() == 0, "Input graph is not symmetric"
      if sp.issparse(adj):
          assert adj.tocsr().max() == 1, "Max value should be 1!"
          assert adj.tocsr().min() == 0, "Min value should be 0!"
      else:
          assert adj.max() == 1, "Max value should be 1!"
          assert adj.min() == 0, "Min value should be 0!"


    def save_adj(self, root=r'/tmp/', name='mod_adj'):
        """
        공격이 들어간 인접행렬을 저장합니다.

        Parameters:
        1. root : 
          설명 : 변수가 저장될 root directory
        2. name : 
          자료형 : str
          설명 : 저장할 파일 이름

        Returns
        -------
        없음

        """
        assert self.modified_adj is not None, \
                'modified_adj is None! Please perturb the graph first.'
        name = name + '.npz'
        modified_adj = self.modified_adj

        if type(modified_adj) is torch.Tensor:
            sparse_adj = utils.to_scipy(modified_adj)
            sp.save_npz(osp.join(root, name), sparse_adj)
        else:
            sp.save_npz(osp.join(root, name), modified_adj)


    def save_features(self, root=r'/tmp/', name='mod_features'):
        """
        공격이 들어간 node feature matrix를 저장합니다.

         Parameters:
        1. root : 
          설명 : 변수가 저장될 root directory
        2. name : 
          자료형 : str
          설명 : 저장할 파일 이름

        Returns
        -------
        없음

        """

        assert self.modified_features is not None, \
                'modified_features is None! Please perturb the graph first.'
        name = name + '.npz'
        modified_features = self.modified_features

        if type(modified_features) is torch.Tensor:
            sparse_features = utils.to_scipy(modified_features)
            sp.save_npz(osp.join(root, name), sparse_features)
        else:
            sp.save_npz(osp.join(root, name), modified_features)

# FGA (Fast Gradient Attack)

In [ ]:
'''     FGA: Fast Gradient Attack on Network Embedding (https://arxiv.org/pdf/1809.02797.pdf) '''
class FGA(BaseAttack):
  """FGA/FGSM.

  Parameters:
  1. model
    설명 : 공격할 모델

  2. nnodes
    자료형 ; int
    설명 : input graph에 있는 노드 개수
  
  3. feature_shape
    자료형 : tuple
    설명 : input node feature의 shape

  4. attack_structure
    자료형 : bool
    설명 : graph structure를 공격할지 말지 정하는 변수

  5. attack_features
    자료형 : bool
    설명 : node features를 공격할지 말지 정하는 변수

  6. device
    자료형 : str
    설명 ; 'cpu' or 'cuda'
  """
  def __init__(self, model, nnodes, feature_shape=None, attack_structure=True, attack_features=False, device='cpu'):

        super(FGA, self).__init__(model, nnodes, attack_structure=attack_structure, attack_features=attack_features, device=device)


        assert not self.attack_features, "not support attacking features"

        if self.attack_features:
            self.feature_changes = Parameter(torch.FloatTensor(feature_shape))
            self.feature_changes.data.fill_(0)
  def attack(self, ori_features, ori_adj, labels, idx_train, target_node, n_perturbations, verbose=False, **kwargs):
        """
        Input graph에 대해 perturbation을 가합니다.

        Parameters:
        1. ori_features
          자료형 : scipy.sparse.csr_matrix
          설명 : 원본(unperturbed) adjacency matrix

        2. ori_adj
          자료형 : scipy.sparse.csr_matrix
          설명 : 원본(unperturbed) node feature matrix

        3. labels 
          설명 : 노드 라벨

        4. idx_train
          설명 : training node 인덱스
        
        5. target_node 
          자료형 : int
          설명 : 공격될 타겟 노드의 인덱스
        
        6. n_perturbations
          자료형 : int
          설명 : Input graph에 들어갈 perturbation 수. Perturbation은
                 간선 추가/삭제 혹은 feature 추가/삭제가 될 수 있다.
        """
        # todense() : 희소행렬을 압축희소행렬로 만들어서 메모리낭비와 연산 시간을 줄임
        modified_adj = ori_adj.todense()
        modified_features = ori_features.todense()
        modified_adj, modified_features, labels = to_tensor(modified_adj, modified_features, labels, device=self.device) # tensor 형태로 반환

        self.surrogate.eval()
        if verbose == True:
            print('number of pertubations: %s' % n_perturbations)

        pseudo_labels = self.surrogate.predict().detach().argmax(1)
        pseudo_labels[idx_train] = labels[idx_train]

        modified_adj.requires_grad = True
        for i in range(n_perturbations):
            adj_norm = normalize_adj_tensor(modified_adj)

            if self.attack_structure:
                output = self.surrogate(modified_features, adj_norm)
                loss = F.nll_loss(output[[target_node]], pseudo_labels[[target_node]])
                grad = torch.autograd.grad(loss, modified_adj)[0]
                # bidirection
                grad = (grad[target_node] + grad[:, target_node]) * (-2*modified_adj[target_node] + 1)
                grad[target_node] = -10
                grad_argmax = torch.argmax(grad)

            value = -2*modified_adj[target_node][grad_argmax] + 1 # target node의 row에 있는 값중 grad가 가장 높은 node값을 이용해 value를 구함
            modified_adj.data[target_node][grad_argmax] += value # 앞서 구한 value값을 양방향으로 더해줌 (undirected : g_ij, g_ji)
            modified_adj.data[grad_argmax][target_node] += value # 양방향으로 더해줌.

            if self.attack_features:
                pass

        modified_adj = modified_adj.detach().cpu().numpy()
        modified_adj = sp.csr_matrix(modified_adj)
        self.check_adj(modified_adj)
        self.modified_adj = modified_adj

# GCN

In [ ]:
class GraphConvolution(Module):
    """
    간단한 GCN layer, https://github.com/tkipf/pygcn 참고
    """

    def __init__(self, in_features, out_features, with_bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if with_bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        """ 
        Forward 함수
        """
        if input.data.is_sparse:
            support = torch.spmm(input, self.weight)
        else:
            support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output


    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'



class GCN(nn.Module):
    """ 
    2 계층 GCN

    Parameters
    ----------
    nfeat : int
        size of input feature dimension
    nhid : int
        number of hidden units
    nclass : int
        size of output dimension
    dropout : float
        dropout rate for GCN
    lr : float
        learning rate for GCN
    weight_decay : float
        weight decay coefficient (l2 normalization) for GCN.
        When `with_relu` is True, `weight_decay` will be set to 0.
    with_relu : bool
        whether to use relu activation function. If False, GCN will be linearized.
    with_bias: bool
        whether to include bias term in GCN weights.
    device: str
        'cpu' or 'cuda'.

    """

    def __init__(self, nfeat, nhid, nclass, dropout=0.5, lr=0.01, weight_decay=5e-4,
            with_relu=True, with_bias=True, device=None):

        super(GCN, self).__init__()

        assert device is not None, "Please specify 'device'!"
        self.device = device
        self.nfeat = nfeat
        self.hidden_sizes = [nhid]
        self.nclass = nclass
        self.gc1 = GraphConvolution(nfeat, nhid, with_bias=with_bias)
        self.gc2 = GraphConvolution(nhid, nclass, with_bias=with_bias)
        self.dropout = dropout
        self.lr = lr
        if not with_relu:
            self.weight_decay = 0
        else:
            self.weight_decay = weight_decay
        self.with_relu = with_relu
        self.with_bias = with_bias
        self.output = None
        self.best_model = None
        self.best_output = None
        self.adj_norm = None
        self.features = None

    def forward(self, x, adj):
        if self.with_relu:
            x = F.relu(self.gc1(x, adj))
        else:
            x = self.gc1(x, adj)

        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

    def initialize(self):
        """
        GCN 파라미터 초기화
        """
        self.gc1.reset_parameters()
        self.gc2.reset_parameters()


    def fit(self, features, adj, labels, idx_train, idx_val=None, train_iters=200, initialize=True, verbose=False, normalize=True, patience=500, **kwargs):
        """
        GCN model 훈련, idx_val이 None이 아니라면 validation loss에 따른 best model을 선택한다.

        Parameters:
        1. features
          설명 : 노드 features
        
        2. adj
          자료형 : torch.tensor 혹은 scipy matrix

        3. labels
          설명 : 노드 라벨
        
        4. idx_train
          설명 : training node 인덱스

        5. idx_val
          설명 : validation node 인덱스. 주어지지 않은 경우 (None 인 경우), GCN 훈련 과정에서 early stopping을 적용하지 않을 예정
        
        6. train_iters
          자료형 : int
          설명 : training epoch 수

        7. initialize
          자료형 : bool
          설명 : training 이전에 파라미터를 초기화할 것인지 말것인지를 결정하는 변수

        8. verbose
          자료형 : bool
          설명 : 상세 모드 할 것인지 말 것인지를 결정

        9. normalize
          자료형 : bool
          설명 : input 인접 행렬을 normalize 할 것인지 말 것인지
        
        10. patience
          자료형 : int
          설명 : early stopping을 위한 patience, idx_val이 주어진 경우에만 유효함.
        """

        self.device = self.gc1.weight.device
        if initialize:
            self.initialize()

        if type(adj) is not torch.Tensor:
            features, adj, labels = to_tensor(features, adj, labels, device=self.device) # utils 없앰 **

        else:
            features = features.to(self.device)
            adj = adj.to(self.device)
            labels = labels.to(self.device)

        if normalize:
            if is_sparse_tensor(adj):
                adj_norm = normalize_adj_tensor(adj, sparse=True)
            else:
                adj_norm = normalize_adj_tensor(adj)
        else:
            adj_norm = adj

        self.adj_norm = adj_norm
        self.features = features
        self.labels = labels

        if idx_val is None:
            self._train_without_val(labels, idx_train, train_iters, verbose)
        else:
            if patience < train_iters:
                self._train_with_early_stopping(labels, idx_train, idx_val, train_iters, patience, verbose)
            else:
                self._train_with_val(labels, idx_train, idx_val, train_iters, verbose)


    def _train_without_val(self, labels, idx_train, train_iters, verbose):
      '''idx_val이 주어지지 않은 경우의 training 과정'''
      self.train()
      optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
      for i in range(train_iters):
          optimizer.zero_grad()
          output = self.forward(self.features, self.adj_norm)
          loss_train = F.nll_loss(output[idx_train], labels[idx_train])
          loss_train.backward()
          optimizer.step()
          if verbose and i % 10 == 0:
              print('Epoch {}, training loss: {}'.format(i, loss_train.item()))

      self.eval()
      output = self.forward(self.features, self.adj_norm)
      self.output = output

    def _train_with_val(self, labels, idx_train, idx_val, train_iters, verbose):
      '''idx_val이 주어진 경우의 train 과정'''
      if verbose:
          print('=== training gcn model ===')
      optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

      best_loss_val = 100
      best_acc_val = 0

      for i in range(train_iters):
          self.train()
          optimizer.zero_grad()
          output = self.forward(self.features, self.adj_norm)
          loss_train = F.nll_loss(output[idx_train], labels[idx_train])
          loss_train.backward()
          optimizer.step()

          if verbose and i % 10 == 0:
              print('Epoch {}, training loss: {}'.format(i, loss_train.item()))

          self.eval()
          output = self.forward(self.features, self.adj_norm)
          loss_val = F.nll_loss(output[idx_val], labels[idx_val])
          # acc_val = utils.accuracy(output[idx_val], labels[idx_val])
          acc_val = accuracy(output[idx_val], labels[idx_val])

          if best_loss_val > loss_val:
              best_loss_val = loss_val
              self.output = output
              weights = deepcopy(self.state_dict())

          if acc_val > best_acc_val:
              best_acc_val = acc_val
              self.output = output
              weights = deepcopy(self.state_dict())

      if verbose:
          print('=== picking the best model according to the performance on validation ===')
      self.load_state_dict(weights)

    def _train_with_early_stopping(self, labels, idx_train, idx_val, train_iters, patience, verbose):
      '''early_stopping을 적용한 training 과정. idx_val이 있는 경우에만 유효하다.'''
      if verbose:
          print('=== training gcn model ===')
      optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

      early_stopping = patience
      best_loss_val = 100

      for i in range(train_iters):
          self.train()
          optimizer.zero_grad()
          output = self.forward(self.features, self.adj_norm)
          loss_train = F.nll_loss(output[idx_train], labels[idx_train])
          loss_train.backward()
          optimizer.step()

          if verbose and i % 10 == 0:
              print('Epoch {}, training loss: {}'.format(i, loss_train.item()))

          self.eval()
          output = self.forward(self.features, self.adj_norm)

            # def eval_class(output, labels):
            #     preds = output.max(1)[1].type_as(labels)
            #     return f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='micro') + \
            #         f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')

            # perf_sum = eval_class(output[idx_val], labels[idx_val])
          loss_val = F.nll_loss(output[idx_val], labels[idx_val])

          if best_loss_val > loss_val:
              best_loss_val = loss_val
              self.output = output
              weights = deepcopy(self.state_dict())
              patience = early_stopping
          else:
              patience -= 1
          if i > early_stopping and patience <= 0:
              break

      if verbose:
           print('=== early stopping at {0}, loss_val = {1} ==='.format(i, best_loss_val) )
      self.load_state_dict(weights)

    def test(self, idx_test):


        """
        test set에 대하여 GCN의 성능을 측정한다.

        Parameters:
        1. idx_test

        설명 : test node 인덱스
        """
        self.eval()
        output = self.predict()
        # output = self.output
        loss_test = F.nll_loss(output[idx_test], self.labels[idx_test])
        # acc_test = utils.accuracy(output[idx_test], self.labels[idx_test])
        acc_test = accuracy(output[idx_test], self.labels[idx_test])
        print("Test set results:",
              "loss= {:.4f}".format(loss_test.item()),
              "accuracy= {:.4f}".format(acc_test.item()))
        return acc_test.item()



    def predict(self, features=None, adj=None):

        """
        Default로는 input은 normalize 되지 않은 인접행렬이다.

        Parameters:
        1. features : 
          설명 : node features. 만약 'features'와 'adj'가 주어지지 않은 경우 이 함수는 이전에 training 할때 저장된 'features'와 'adj'를 사용하여 prediction을 할 것임

        2. adj
          설명 : 인접 행렬. 만약 'features'와 'adj'가 주어지지 않은 경우 이 함수는 이전에 training 할때 저장된 'features'와 'adj'를 사용하여 prediction을 할 것임

        
        Returns :

        1. torch.FloatTensor
           설명 :  output (log probabilities) of GCN
        """

        self.eval()
        if features is None and adj is None:
            return self.forward(self.features, self.adj_norm)
        else:
            if type(adj) is not torch.Tensor:
                # features, adj = utils.to_tensor(features, adj, device=self.device)
                features, adj = to_tensor(features, adj, device=self.device)

            self.features = features
            # if utils.is_sparse_tensor(adj):
            if is_sparse_tensor(adj):
                # self.adj_norm = utils.normalize_adj_tensor(adj, sparse=True)
                self.adj_norm = normalize_adj_tensor(adj, sparse=True)
            else:
                # self.adj_norm = utils.normalize_adj_tensor(adj)
                self.adj_norm = normalize_adj_tensor(adj)
            return self.forward(self.features, self.adj_norm)

# Attack 코드

In [ ]:
data_name = 'cora' # args.dataset을 data_name으로 지정해줄 예정 **
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

np.random.seed(777)
torch.manual_seed(777)
torch.cuda.manual_seed(777)

data = Dataset(root='/tmp/',name=data_name) # data_name으로 바꿔줌 **
adj, features, labels = data.adj, data.features, data.labels # data(cora)의 adj, features, labels 정보를 받아옴
idx_train, idx_val, idx_test = data.idx_train, data.idx_val, data.idx_test # data의 index 정보를 받아옴

idx_unlabeled = np.union1d(idx_val, idx_test) # unlabeled 된 index 정보를 받아옴

# Setup Surrogate model
surrogate = GCN(nfeat=features.shape[1], nclass=labels.max().item()+1,
                nhid=16, device=device)

surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train, idx_val)

# Setup Attack Model
target_node = 0
model = FGA(surrogate, nnodes=adj.shape[0], device=device)
model = model.to(device)

Loading cora dataset...
Done!
Selecting 1 largest connected components


In [ ]:
def main():
    u = 0 # node to attack
    assert u in idx_unlabeled # target node는 train과정에 있으면 안됨 

    degrees = adj.sum(0).A1 
    n_perturbations = int(degrees[u]) # perturbation 을 얼마나 많이 할 것인지. Default : 노드의 degree 값

    model.attack(features, adj, labels, idx_train, target_node, n_perturbations)

    print('=== testing GCN on original(clean) graph ===')
    test(adj, features, target_node)

    print('=== testing GCN on perturbed graph ===')
    test(model.modified_adj, features, target_node)

In [ ]:
def test(adj, features, target_node):
    ''' test on GCN '''
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj, labels, idx_train)

    gcn.eval()
    output = gcn.predict()
    probs = torch.exp(output[[target_node]])[0]
    print('probs: {}'.format(probs.detach().cpu().numpy()))
    acc_test = accuracy(output[idx_test], labels[idx_test])

    print("Test set results:",
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()


In [ ]:
def select_nodes(target_gcn=None):
    '''
    nettack 논문에 쓰인 방식대로 node를 선택한다.
    1. 가장 높은 classification margin값을 가지는 10개의 node (clearly correctly classified)
    2. 가장 낮은 classification margin 값을 가지는 10개의 node(여전히 올바르게 분류됨)
    3. 20개의 노드를 추가로 랜덤하게
    '''

    if target_gcn is None:
        target_gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)
        target_gcn = target_gcn.to(device)
        target_gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
    target_gcn.eval()
    output = target_gcn.predict()

    margin_dict = {}
    for idx in idx_test:
        margin = classification_margin(output[idx], labels[idx])
        if margin < 0: # margin이 0 이하인 node는 고려하지 않음(올바르게 분류된 노드만 고려)
            continue
        margin_dict[idx] = margin
    sorted_margins = sorted(margin_dict.items(), key=lambda x:x[1], reverse=True)
    high = [x for x, y in sorted_margins[: 10]]
    low = [x for x, y in sorted_margins[-10: ]]
    other = [x for x, y in sorted_margins[10: -10]]
    other = np.random.choice(other, 20, replace=False).tolist()

    return high + low + other

In [ ]:
def multi_test_poison():
    # poisoning attack 환경에서 40개의 node에 대해 test
    cnt = 0
    degrees = adj.sum(0).A1
    node_list = select_nodes()
    num = len(node_list)
    print('=== [Poisoning] Attacking %s nodes respectively ===' % num)
    for target_node in tqdm(node_list):
        n_perturbations = int(degrees[target_node])
        model = FGA(surrogate, nnodes=adj.shape[0], device=device)
        model = model.to(device)
        model.attack(features, adj, labels, idx_train, target_node, n_perturbations)
        modified_adj = model.modified_adj
        acc = single_test(modified_adj, features, target_node)
        if acc == 0:
            cnt += 1
    print('misclassification rate : %s' % (cnt/num))

In [ ]:
def single_test(adj, features, target_node, gcn=None):
    if gcn is None:
        # poisoning attack을 GCN에서 test
        gcn = GCN(nfeat=features.shape[1],
                  nhid=16,
                  nclass=labels.max().item() + 1,
                  dropout=0.5, device=device)

        gcn = gcn.to(device)

        gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)
        gcn.eval()
        output = gcn.predict()
    else:
        # evasion attack을 GCN에서 test
        output = gcn.predict(features, adj)
    probs = torch.exp(output[[target_node]])

    # acc_test = accuracy(output[[target_node]], labels[target_node])
    acc_test = (output.argmax(1)[target_node] == labels[target_node])
    return acc_test.item()

In [ ]:
def multi_test_evasion():
    # evasion attack을 40개의 node에서 test
    # target_gcn = GCN(nfeat=features.shape[1],
    #           nhid=16,
    #           nclass=labels.max().item() + 1,
    #           dropout=0.5, device=device)

    # target_gcn = target_gcn.to(device)
    # target_gcn.fit(features, adj, labels, idx_train, idx_val, patience=30)

    target_gcn = surrogate
    cnt = 0
    degrees = adj.sum(0).A1
    node_list = select_nodes(target_gcn)
    num = len(node_list)

    print('=== [Evasion] Attacking %s nodes respectively ===' % num)
    for target_node in tqdm(node_list):
        n_perturbations = int(degrees[target_node])
        model = FGA(surrogate, nnodes=adj.shape[0], device=device)
        model = model.to(device)
        model.attack(features, adj, labels, idx_train, target_node, n_perturbations)
        modified_adj = model.modified_adj

        acc = single_test(modified_adj, features, target_node, gcn=target_gcn)
        if acc == 0:
            cnt += 1 # accuracy 값이 0인걸 세어서 misclassification rate 계산에 이용
    print('misclassification rate : %s' % (cnt/num))

In [ ]:
if __name__ == '__main__':
    main()
    multi_test_evasion()
    multi_test_poison()

=== testing GCN on original(clean) graph ===
probs: [9.5474003e-03 6.7936694e-03 9.1732377e-03 6.4560480e-02 5.0589041e-04
 8.9606100e-01 1.3358367e-02]
Test set results: accuracy= 0.8300
=== testing GCN on perturbed graph ===
probs: [1.08786386e-04 1.94418699e-05 9.07947979e-05 7.05818311e-05
 1.56384442e-06 1.00363675e-03 9.98705149e-01]
Test set results: accuracy= 0.8325
=== [Evasion] Attacking 40 nodes respectively ===


100%|██████████| 40/40 [00:24<00:00,  1.61it/s]


misclassification rate : 0.975
=== [Poisoning] Attacking 40 nodes respectively ===


100%|██████████| 40/40 [00:34<00:00,  1.15it/s]

misclassification rate : 0.85
